# NER модель 
# для удаления названий компаний из текстов

Названия компаний, в основном, это идентификаторы на которых переобучается модель. Разработаем автоматический фильтр названий компаний для предобработки набора данных.

Сделаем объединение результатов с двух моделей: эвристической и BERT.

## Модель1 Эвристические правила

Простая модель основана на эвристрических правилах обработки текстовых строк.

In [49]:
import re

class NERFilter:
    def __init__(self):
        self._lowFreqWords = []

    # Удаляем подстроки с большой буквы внутри кавычек «»
    def _ruleDoubleAngleQuotation(self, s, do_filter = True):
        result = s
        pattern = '[ (]«[А-ЯЁA-Z][^«»]*»'
        matches = re.findall(pattern, s)

        if do_filter:
            for m in matches:
                result = result.replace(m, '')

            return result
        else:
            return matches
    
    # Удаляем низкочастотные подстроки с большой буквы
    def _ruleLowFreqsWithCaps(self, s):
        result = []
        for w in s.split(' '):
            if w not in self._lowFreqWords:
                result.append(w)

        return ' '.join(result)
    
     
    def fit(self, corpus, importantWords, level = 10):
        """
        Args:
            corpus — список строк;
            importantWords — список слов для сохранения (такие слова не будут удаляться);
            level — уровень кол-ва появления слова в текстах, чтобы его отфильтровать 
                    (слова с большой буквы с частотностью ниже level будут удалены
                    при вызове метода filter).
        """
        freqs = {}

        # Get frequencies of words:
        for txt in corpus:
            for w in txt.split(' '):
                #print('"'+w+'"')
                if len(w) > 1 \
                        and w[0].upper() == w[0] \
                        and w.lower() not in importantWords \
                        and '-' not in w:
                    if w in freqs:
                        freqs[w] = freqs[w] + 1
                    else:
                        freqs[w] = 1

        # Write results:
        
        for k, v in freqs.items():
            if v < level:
                self._lowFreqWords.append(k)

        return self._lowFreqWords
    

    def filter(self, s):
        """
        Метод для фильтрации названий из текста.
        """
        result = self._ruleDoubleAngleQuotation(s)
        result = self._ruleLowFreqsWithCaps(result)
        return result.replace('  ', ' ')
    
    def get_ner_results(self, s):
        results = {}

        for m in self._ruleDoubleAngleQuotation(s, False):
            results[m] = True

        for w in self._lowFreqWords:
            results[w] = True
    
        return list(map(lambda it: (it, -1, -1), results))

In [4]:
txt = "IVAT it  IVA Technologies - один из крупных игроков российского IT-рынка разработчик экосистемы корпоративных коммуникаций IVA.  В экосистему IVA входят: корпоративный мессенджер IVA Connect платформа видеоконференцсвязи IVA MCU облачная платформа видеоконференцсвязи «ВКурсе» помощник основанный на технологии искусственного интеллекта IVA GPT система управления и мониторинга IVA MS сервер корпоративной телефонии IVA CS серия IP-телефонов видеотерминалы IVA Room и IVA Largo пограничный контроллер сессий IVA SBC и другие продукты."

In [50]:
corpus = [txt,]
importantWords = ['gpt',]

nerModel = NERFilter()
nerModel.fit(corpus, importantWords, level = 15)
heuristic_results = nerModel.get_ner_results(txt)
print('Entities:\n', heuristic_results)
print('Filtered text:\n', nerModel.filter(txt))


Entities:
 [(' «ВКурсе»', -1, -1), ('IVAT', -1, -1), ('IVA', -1, -1), ('Technologies', -1, -1), ('IVA.', -1, -1), ('Connect', -1, -1), ('MCU', -1, -1), ('«ВКурсе»', -1, -1), ('MS', -1, -1), ('CS', -1, -1), ('Room', -1, -1), ('Largo', -1, -1), ('SBC', -1, -1)]
Filtered text:
 it - один из крупных игроков российского IT-рынка разработчик экосистемы корпоративных коммуникаций В экосистему входят: корпоративный мессенджер платформа видеоконференцсвязи облачная платформа видеоконференцсвязи помощник основанный на технологии искусственного интеллекта GPT система управления и мониторинга сервер корпоративной телефонии серия IP-телефонов видеотерминалы и пограничный контроллер сессий и другие продукты.


Мы получили NER модель, которую можно использовать для предобработки текстов описаний компаний в развитом ПО, которое будет использовать ML модель.

## Модель2 NER BERT

Используем предобученную модель BERT based: https://huggingface.co/webmichaelnosenko/rust-bert-base-NER

In [1]:
from transformers import pipeline

In [51]:
def join_results(bert_results, txt):
    results = []

    if len(bert_results) > 0:
        start = bert_results[0]['start']
        end = bert_results[0]['end']
        for result in bert_results:
            if result['start'] - end > 1:
                results.append((txt[start:end + 1], start, end))
                start = result['start']
            
            if txt[result['end']].isalpha():
                end = result['end']
            else:
                end = result['end'] - 1

        results.append((txt[start:end + 1], start, end))

    return results


def get_bert_results(txt, level_score, verbose = True):
    ner_pipe = pipeline("ner", model="webmichaelnosenko/rust-bert-base-NER", device='cuda')

    results = ner_pipe(txt)

    results = list(filter(lambda it: it['score'] > level and 'ORG' in it['entity'], results))

    if verbose:
        for result in results:
                print(result)

    processed_results = join_results(results, txt)

    return processed_results


def filter_by_bert_ner_results(ner_results, txt):
    result = list(txt)
    null_character = ''

    for it in ner_results:
        for i in range(it[1], it[2] + 1):
             result[i] = null_character

    result = "".join(result).replace(null_character, '')
    return result


level_score = 0.5 # Используем результаты со score больше заданного уровня.
bert_results = get_bert_results(txt, level_score)
print('Entities:\n', bert_results)

print('Filtered text:\n', filter_by_bert_ner_results(bert_results, txt))

Some weights of the model checkpoint at webmichaelnosenko/rust-bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'entity': 'B-ORG', 'score': 0.9943851, 'index': 4, 'word': 'IV', 'start': 9, 'end': 11}
{'entity': 'I-ORG', 'score': 0.99040914, 'index': 5, 'word': '##A', 'start': 11, 'end': 12}
{'entity': 'I-ORG', 'score': 0.9776227, 'index': 6, 'word': 'Technologies', 'start': 13, 'end': 25}
{'entity': 'I-ORG', 'score': 0.9517372, 'index': 89, 'word': 'IV', 'start': 123, 'end': 125}
{'entity': 'I-ORG', 'score': 0.8818024, 'index': 90, 'word': '##A', 'start': 125, 'end': 126}
{'entity': 'B-ORG', 'score': 0.679242, 'index': 103, 'word': 'IV', 'start': 142, 'end': 144}
{'entity': 'I-ORG', 'score': 0.6667436, 'index': 104, 'word': '##A', 'start': 144, 'end': 145}
{'entity': 'B-ORG', 'score': 0.63552064, 'index': 339, 'word': 'IP', 'start': 429, 'end': 431}
{'entity': 'I-ORG', 'score': 0.6018182, 'index': 363, 'word': '##A', 'start': 459, 'end': 460}
{'entity': 'B-ORG', 'score': 0.83839786, 'index': 366, 'word': 'IV', 'start': 468, 'end': 470}
{'entity': 'I-ORG', 'score': 0.6148589, 'index': 367, 'word

## Объединим обе модели

In [57]:
def combine_results(model1_results, model2_results):
    combined_results = []
    res1 = list(map(lambda it: it[0], model1_results))
    res2 = list(map(lambda it: it[0], model2_results))

    for it in res1:
        if it in res2:
            combined_results.append(it)

    return combined_results


def filter_by_combined_results(ner_results, txt):
    result = txt
    for it in ner_results:
        result = result.replace(it, '')

    return result

In [60]:
combined_results = combine_results(heuristic_results, bert_results)
print('Entities:\n', combined_results)
print('Filtered text:\n', filter_by_combined_results(combined_results, txt))

Entities:
 ['IVA', 'Technologies']
Filtered text:
 T it    - один из крупных игроков российского IT-рынка разработчик экосистемы корпоративных коммуникаций .  В экосистему  входят: корпоративный мессенджер  Connect платформа видеоконференцсвязи  MCU облачная платформа видеоконференцсвязи «ВКурсе» помощник основанный на технологии искусственного интеллекта  GPT система управления и мониторинга  MS сервер корпоративной телефонии  CS серия IP-телефонов видеотерминалы  Room и  Largo пограничный контроллер сессий  SBC и другие продукты.


## Выводы

На прилагающемся скриншоте <a href="NERResults.png">NERResults.png</a> мы видим результаты фильтрации текста из примера:
- Слева Модель1 (работающая по эвристическим правилам). Модель агрессивно вычистила из текста не только название организации, но и все названия продуктов, но не тронула термины, например: «IP-телефонов», «IT-рынка».
- Справа Модель2 (BERT based). Модель удалила отдельные названия компании, но оставила артефакты: в названиях продуктов содержится и название компании, например: «IVA Connect». Также, модель удалила термин "IP-".
- В середине комбинированные результаты. Результаты сбалансированные: удалены все упоминания компании, но остались термины и названия продуктов.

Пользователь может выбрать нужный вариант NER модели в зависимости от требований к производительности решения.